In [21]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os # accessing directory structure

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten




In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)


In [23]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_057b90874d074858851a6b72b9401168 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_057b90874d074858851a6b72b9401168 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_057b90874d074858851a6b72b9401168 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='oDkAOEnFOplGq4Pd0ctBKWKpnkOkPXayNGFWtl3CJh9L',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_057b90874d074858851a6b72b9401168)

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_10 = client_057b90874d074858851a6b72b9401168.get_object(Bucket='fashionassistant-donotdelete-pr-mjgt9bn5i8dhxd', Key='ibmproj1.zip')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_10, "__iter__"): streaming_body_10.__iter__ = types.MethodType( __iter__, streaming_body_10 ) 


In [24]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_10.read()),'r')
file_paths=unzip.namelist()
#print(file_paths)
for path in file_paths:
    unzip.extract(path) 

In [25]:
pwd

'/home/wsuser/work'

In [26]:
import os
filenames=os.listdir('/home/wsuser/work/ibmproj/train')

In [27]:
x_train=train_datagen.flow_from_directory('/home/wsuser/work/ibmproj/train',target_size=(64,64),batch_size=32,class_mode="categorical")
x_test=test_datagen.flow_from_directory('/home/wsuser/work/ibmproj/test',target_size=(64,64),batch_size=32,class_mode="categorical")

Found 4108 images belonging to 7 classes.
Found 2215 images belonging to 7 classes.


In [28]:
x_train.class_indices

{'accessories': 0,
 'bags': 1,
 'footwear': 2,
 'mens wear': 3,
 'sunglasses': 4,
 'watches': 5,
 'womens wear': 6}

In [29]:
model=Sequential()

In [30]:
model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation="relu"))

In [31]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [32]:
model.add(Convolution2D(64,(3,3),activation="relu"))

In [33]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [34]:
model.add(Convolution2D(128,(3,3),activation="relu"))

In [35]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [36]:
model.add(Flatten()) #input layer 

In [37]:
model.add(Dense(units=7,kernel_initializer="random_uniform",activation="relu"))

In [38]:
model.add(Dense(units=7,kernel_initializer="random_uniform",activation="softmax"))

In [39]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [41]:
model.fit_generator(x_train,steps_per_epoch =128 ,epochs=50,validation_data=x_test,validation_steps=69)

Epoch 1/50
128/128 [==============================] - 184s 1s/step - loss: 0.5012 - accuracy: 0.8261 - val_loss: 0.5760 - val_accuracy: 0.7966
Epoch 2/50
128/128 [==============================] - 186s 1s/step - loss: 0.4688 - accuracy: 0.8449 - val_loss: 0.4798 - val_accuracy: 0.8229
Epoch 3/50
128/128 [==============================] - 186s 1s/step - loss: 0.4170 - accuracy: 0.8624 - val_loss: 0.4908 - val_accuracy: 0.8279
Epoch 4/50
128/128 [==============================] - 186s 1s/step - loss: 0.3808 - accuracy: 0.8685 - val_loss: 0.4338 - val_accuracy: 0.8424
Epoch 5/50
128/128 [==============================] - 185s 1s/step - loss: 0.3380 - accuracy: 0.8881 - val_loss: 0.4445 - val_accuracy: 0.8438
Epoch 6/50
128/128 [==============================] - 185s 1s/step - loss: 0.3213 - accuracy: 0.8913 - val_loss: 0.4598 - val_accuracy: 0.8519
Epoch 7/50
128/128 [==============================] - 186s 1s/step - loss: 0.2977 - accuracy: 0.9053 - val_loss: 0.4023 - val_accuracy: 0.8537

In [42]:
model.save("fashion.h5")

In [43]:
!tar -zcvf image-classification-model_new.tgz fashion.h5

fashion.h5


In [44]:
ls -1

fashion.h5
ibmproj/
image-classification-model_new.tgz


In [45]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 27.8 MB/s eta 0:00:01


In [46]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"pwNooTwaTrFqobYoOD4ITR71Xn5GP0ccE5kVbPuQ2vbj"
}
client =APIClient(wml_credentials)

In [47]:
client=APIClient(wml_credentials)

In [48]:
def guide_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])


In [49]:
space_uid=guide_from_space_name(client,'imageclassification')
print("Space UID = "+ space_uid)

Space UID = 8183ee11-bfbb-49c3-b32b-07d6c488c0ae


In [50]:
client.set.default_space(space_uid)

'SUCCESS'

In [51]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [52]:
software_spec_uid=client.software_specifications.get_uid_by_name('tensorflow_1.15-py3.6')
software_spec_uid

'2b73a275-7cbf-420b-a912-eae7f436e0bc'

In [53]:
model_details=client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
client.repository.ModelMetaNames.NAME:"imageclassification",
client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid }
                                           )

model_id=client.repository.get_model_uid(model_details)


Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [54]:
model_id

'f0257b2a-c811-490f-8ce4-8177211925df'

In [55]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'

In [56]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [57]:
model=load_model("fashion.h5")

In [114]:

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_18 = client_057b90874d074858851a6b72b9401168.get_object(Bucket='fashionassistant-donotdelete-pr-mjgt9bn5i8dhxd', Key='mens.zip')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_18, "__iter__"): streaming_body_18.__iter__ = types.MethodType( __iter__, streaming_body_18 ) 


In [115]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_18.read()),'r')
file_paths=unzip.namelist()
#print(file_paths)
for path in file_paths:
    unzip.extract(path) 


In [116]:
img=image.load_img(path,streaming_body_18,target_size=(64,64))

In [117]:
import numpy as np
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)

In [118]:
pred=model.predict_classes(x)

In [119]:
pred[0]

3

In [120]:
index=["accessories","bags","footwear","mens wear","sunglasses","watches","womens wear"]

In [121]:
a=index[pred[0]]
print(a)

mens wear
